<a href="https://colab.research.google.com/github/karl-karlsson/notebooks/blob/main/laptops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You can open a copy of this notebook in Google Colab by clicking the icon above

-----

![laptop](https://freesvg.org/img/1520464168.png)

# Querying inventory using Pyhon

This was originally a guided project from Dataquest ([link](https://www.dataquest.io/)) although I've made some tweaks to it.

I've downloaded a dataset on laptops to my GitHub respository from Kaggle ([info about dataset](https://www.kaggle.com/datasets/ionaskel/laptop-prices)).

Since this orignally is a *guided* project I don't want to take full credit for the solutions or the framing of the project even though I've made some modifications and implemented some own functionalities. This project have been primarily for my own learning experience and a bit of fun.

# Scenario
Let's imagine that we own a computer retailer that sells laptops and we want to build a way to answer different business questions about our inventory. The `laptops.csv` file is our inventory and is stored online.

The goal of the project is to create a new class to represent our inventory. When a new object (of type `Inventory` is initiated) it will automatically load the .csv file from the web (using the url supplied as input). 

I'll also define methods that belongs to the new class. These methods will help us answer questions such as:

* Given a laptop id, find the corresponding data.
* Get total number of laptops in stocks
* Find the cheapest and the most expensive laptops
* Find the most expensive laptops whose price falls within a given budget.

In the first step I'll:
* Download the file from GitHub using the `requests` package
* Read in the file using the `csv` package 
* Print out the column names and the first five rows 



In [4]:
import csv
import requests

url = "https://raw.githubusercontent.com/karl-karlsson/data/main/laptops.csv"


with requests.Session() as s:
    download = s.get(url)

    decoded_content = download.content.decode('latin-1')

    csv_read_file = csv.reader(decoded_content.replace('"','').splitlines()) 
    a_list = list(csv_read_file)

print("Column names")
print(a_list[0])
print("\n")
print("Inventory")
for row in range(1,6):
  print(a_list[row])


Column names
['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price_euros']


Inventory
['1', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339.69']
['2', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898.94']
['3', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575.00']
['4', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537.45']
['5', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB',

I'll start by doing the following:

* Create a new `Inventory` class that takes the url as input and reads in the .csv file
* Define three methods:
  * `attributes` that prints out the column names
  * `stock` that returns the current number of laptops (all rows exept column names) in stock
  * `data` that returns all rows, except the column names

I will build out the `Inventory` class with more methods for more advanced operations further down

In [21]:
class Inventory():                    
    
    def __init__(self, url): 
      with requests.Session() as s:
          download = s.get(url)

          decoded_content = download.content.decode('latin-1')

          csv_read_file = csv.reader(decoded_content.splitlines(), delimiter=',')
          data = list(csv_read_file)

          self.attributes = data[:1]
          self.data = data[1:]
          self.stock = len(data[1:])
                  


Let's create a new `Inventory` instance named `laptops` from the url used before.

Let´s print out the attributes of laptops by calling the `laptops.attributes` method on `laptops`


In [22]:
laptops_url = 'https://raw.githubusercontent.com/karl-karlsson/data/main/laptops.csv' 

laptops = Inventory(laptops_url)  

print(laptops.attributes)

[['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price_euros']]


We can check how many laptops are currently in stock by calling the `stock` method on `laptops`

In [23]:
print("Current stock:", 
      laptops.stock, 
      "laptops")

Current stock: 1303 laptops


Actually, the above code is a bit tidious to write if you want to query the current stock multiple times. It's perhaps better to define a new method (`get_stock()`) outside of the `__init__` constructor. 

I also want to define a new method (`get_id`) that takes an id as input and returns the corresponding laptop. This could be convenient if a customer customer comes to our store with a purchase slip because then we can quickly identify the laptop to which it corresponds.

In [33]:
class Inventory():                    
    
    def __init__(self, url): 
      with requests.Session() as s:
          download = s.get(url)

          decoded_content = download.content.decode('latin-1')

          csv_read_file = csv.reader(decoded_content.replace('"','').splitlines()) 
          data = list(csv_read_file)
          self.attributes = data[:1]
          self.data = data[1:]
          for row in self.data:
            row[0] = int(row[0]) # Convert ID to integer
          for row in self.data:              
            row[-1] = int(float(row[-1])) # Convert price_euros to integer
          self.stock = len(data[1:])

    # NEW! get_stock() method
    def get_stock(self) :
      print("Current stock is: ", self.stock, " laptops")

    # NEW! get_id() method
    def get_id(self, laptop_id):  
        for row in self.data:                  
            if row[0] == laptop_id:
                return row
        print("No laptop with Id = ", laptop_id)                           

Let's see if we can load the inventory data (as an `Inventory` object) and use the new `get_stock` method.

In [34]:
# Reload the data
laptops = Inventory(laptops_url)

laptops.get_stock()

Current stock is:  1303  laptops


Works like a charm! Let's see if we can make queries about laptops with Id:s 25 (exists), 215 (exists) and 2043 (doesn't exist)

In [31]:
# Reload the data
laptops = Inventory(laptops_url)

# Get laptop with ID = 25 ?
laptops.get_id(25)

[25,
 'HP',
 '15-BS101nv (i7-8550U/8GB/256GB/FHD/W10)',
 'Ultrabook',
 '15.6',
 'Full HD 1920x1080',
 'Intel Core i7 8550U 1.8GHz',
 '8GB',
 '256GB SSD',
 'Intel HD Graphics 620',
 'Windows 10',
 '1.91kg',
 659]

In [27]:
# Get laptop with ID = 215 ?
laptops.get_id(215)

[215,
 'Acer',
 'Aspire 7',
 'Notebook',
 '15.6',
 'Full HD 1920x1080',
 'Intel Core i7 7700HQ 2.8GHz',
 '8GB',
 '1TB HDD',
 'Nvidia GeForce GTX 1050',
 'Linux',
 '2.4kg',
 779]

In [29]:
# Get laptop with ID = 2043 (doesn't exist) ?
laptops.get_id(2043)

No laptop with Id =  2043


Works as intented.

Let's now say that we have four types of customers.
Customers that prefer cheap laptops, customers that prefer expensive laptops, customers who don't know what they want and customers who prefers Apple.

To serve our different customers better (and faster) we want four new methods for our Inventory class.

`Inventory.get_cheap()` that returns the cheapest laptop

`Inventory.get_expensive()` that returns the most expensive laptop 

`Inventory.get_mix()` that returns the cheapest, the most expensive and the middle (median) priced laptop

`Inventory.get_budget(budget=1000)` that returns the most expensive laptop that satisfy the customers budget constraint (default is 1000 euro).

`Inventory.get_apple()` that returns a random Apple laptop 

In [146]:
class Inventory():                    
    
    def __init__(self, url): 
      with requests.Session() as s:
          download = s.get(url)

          decoded_content = download.content.decode('latin-1')

          csv_read_file = csv.reader(decoded_content.replace('"','').splitlines()) 
          data = list(csv_read_file)
          self.attributes = data[:1]
          self.data = data[1:]
          for row in self.data:
            row[0] = int(row[0]) 
          for row in self.data:              
            row[-1] = int(float(row[-1])) 
          self.stock = len(data[1:])

    def get_stock(self) :
      print("Current stock is: ", self.stock, " laptops")

    def get_id(self, laptop_id):  
        for row in self.data:                  
            if row[0] == laptop_id:
                return row
        print("No laptop with Id = ", laptop_id)

    # NEW! get_cheap() method
    def get_cheap(self) :
      cheapest = self.data[0][-1]
      for row in self.data[1:] :
          if row[-1] < cheapest :
              cheapest = row[-1]
      for row in self.data :
        if row[-1] == cheapest :
          return row

    # NEW! get_expensive() method
    def get_expensive(self) :
      expensive = self.data[0][-1]
      for row in self.data[1:] :
        if row[-1] > expensive :
          expensive = row[-1]
      for row in self.data :
          if row[-1] == expensive :
            return row

    # NEW! get_mix() method
    def get_mix(self) :
      prices = []
      for row in self.data :
        prices.append(row[-1])
      prices = sorted(prices)
      if len(prices) % 2 == 0:  
        median = (prices[len(prices) // 2] + prices[(len(prices) - 1) // 2]) / 2
      else:
        median = prices[len(prices) // 2]
      laptop_mix = []
      laptop_mix.append(self.get_cheap())
      for row in self.data :
          if row[-1] == median :
              laptop_mix.append(row)
              break 
      laptop_mix.append(self.get_expensive())
      return laptop_mix
    
    # NEW! get_budget() method
    def get_budget(self, budget=1000) :
      prices = []
      for row in self.data :
        prices.append(row[-1])
      prices = sorted(prices)
      best = 0
      for price in prices :
        if price>best and price<=budget :
          best = price
      for row in self.data :
        if row[-1] == best :
          return row

    # NEW! get_apple() method
    def get_apple(self) :
      import random
      index = list(range(0,len(self.data)))
      random.shuffle(index)
      for i in index :
        row = self.data[i]
        if row[1] == "Apple" :
          return row
          break


Okay. So now we have a couple of convenient methods for our class (`Inventory`)Lets start by initiating a new inventory and call the `Inventory.get_cheap()` method. 

In [140]:
# Reload the data
laptops = Inventory(laptops_url)

# get_cheap()
laptops.get_cheap()

[1233,
 'Acer',
 'C740-C9QX (3205U/2GB/32GB/Chrome',
 'Netbook',
 '11.6',
 '1366x768',
 'Intel Celeron Dual Core 3205U 1.5GHz',
 '2GB',
 '32GB SSD',
 'Intel HD Graphics',
 'Chrome OS',
 '1.3kg',
 174]

The cheapest laptop in stock is an Acer that costs 174 euro.

Lets call the `Inventory.get_expensive()` method next.

In [141]:
# get_expensive()
laptops.get_expensive()

[200,
 'Razer',
 'Blade Pro',
 'Gaming',
 '17.3',
 '4K Ultra HD / Touchscreen 3840x2160',
 'Intel Core i7 7820HK 2.9GHz',
 '32GB',
 '1TB SSD',
 'Nvidia GeForce GTX 1080',
 'Windows 10',
 '3.49kg',
 6099]

The most expensive laptop is a Razer laptop with a powerful graphics card and 32 GB of ram. Price? 6099 euro.

Now we'll try the `Inventory.get_mix()` method

In [142]:
# get_mix()
laptops.get_mix()

[[1233,
  'Acer',
  'C740-C9QX (3205U/2GB/32GB/Chrome',
  'Netbook',
  '11.6',
  '1366x768',
  'Intel Celeron Dual Core 3205U 1.5GHz',
  '2GB',
  '32GB SSD',
  'Intel HD Graphics',
  'Chrome OS',
  '1.3kg',
  174],
 [81,
  'HP',
  'ProBook 470',
  'Notebook',
  '17.3',
  'Full HD 1920x1080',
  'Intel Core i5 8250U 1.6GHz',
  '8GB',
  '128GB SSD +  1TB HDD',
  'Nvidia GeForce 930MX',
  'Windows 10',
  '2.5kg',
  977],
 [200,
  'Razer',
  'Blade Pro',
  'Gaming',
  '17.3',
  '4K Ultra HD / Touchscreen 3840x2160',
  'Intel Core i7 7820HK 2.9GHz',
  '32GB',
  '1TB SSD',
  'Nvidia GeForce GTX 1080',
  'Windows 10',
  '3.49kg',
  6099]]

Make your pick. Or do you prefer Apple? No problem with `Inventory.get_apple()`

In [143]:
# get_apple() 
laptops.get_apple()

[5,
 'Apple',
 'MacBook Pro',
 'Ultrabook',
 '13.3',
 'IPS Panel Retina Display 2560x1600',
 'Intel Core i5 3.1GHz',
 '8GB',
 '256GB SSD',
 'Intel Iris Plus Graphics 650',
 'macOS',
 '1.37kg',
 1803]

In [144]:
# get_apple() again
laptops.get_apple()

[1228,
 'Apple',
 'MacBook 12',
 'Ultrabook',
 '12',
 'IPS Panel Retina Display 2304x1440',
 'Intel Core M 1.2GHz',
 '8GB',
 '512GB Flash Storage',
 'Intel HD Graphics 515',
 'Mac OS X',
 '0.920kg',
 1279]

Everytime we call `Inventory.get_apple()` we get a random Apple laptop.

Finally, how about a customer that wants to spend a maximum of 2000 euro?

In [145]:
# get_budget()
laptops.get_budget(2000)

[435,
 'HP',
 'Omen 17-w207nv',
 'Gaming',
 '17.3',
 'Full HD 1920x1080',
 'Intel Core i7 7700HQ 2.8GHz',
 '12GB',
 '256GB SSD +  1TB HDD',
 'Nvidia GeForce GTX 1070',
 'Windows 10',
 '3.35kg',
 1999]

1999 euro is less than 2000 so this is within budget.

# Summary

In this project I created a new class (`Inventory`) that took a url as input and read in an inventory (.csv) file

I also created different methods for the new class that:
* Returned the total number of laptops in stock
* Given an Id return the matching laptop
* Returned the cheapest, most expensive and the middle-priced (median) laptop
* Given a budget returned the most expensive laptop within the budget
* Returned a random Apple computer

That's all folks. 


# Contact details

![bild](https://avatars.githubusercontent.com/u/99097833?v=4)

Feel free to reach out:

* Twitter: [@vaxjo_kalle](https://twitter.com/vaxjo_kalle) 
* LinkedIn: [My profile](https://www.linkedin.com/in/karlkarlssonvaxjo/)
* E-mail: [karl_karlsson@icloud.com](mailto:karl_karlsson@icloud.com)
* GitHub: [github.com/karl-karlsson](https://github.com/karl-karlsson)